In [ ]:
import numpy as np
import math
import scipy
import pandas as pd
import PIL
import gdal
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import sys, os
from pathlib import Path
import time
import random
import collections, functools, operator
import csv
import subprocess
import datetime

from osgeo import gdal,osr
from gdalconst import *
import subprocess
from osgeo.gdalconst import GA_Update

In [ ]:
def viz_raster(rBand, gBand, bBand):
  img = np.dstack((rBand,gBand,bBand))
  plt.figure(num=None, figsize=(15, 12), dpi=80, facecolor='w', edgecolor='k')
  plt.imshow(img)
  plt.show()


def band_max_values(bandList, m = 8., verbose = False):
  '''
  Finds outliers value pixels from bands and limits the max possible value of that band to the median of its neighbours
  '''
  # Get max values from each band
  maxs = []
  for band in bandList:
    if isinstance(band, gdal.Dataset):
      band = band.ReadAsArray() # To numpy array

    maxs.append(np.max(band))
  
  # Fix outlier pixels
  if verbose:
    print('Input:')
    plt.figure(1)
    plt.plot(maxs)
    plt.show()
  
  maxs = np.array(maxs)
  d = np.abs(maxs - np.median(maxs))
  mdev = np.median(d)
  s = d/mdev if mdev else 0.
  outlierBands = np.where(s>m)[0]

  for band in outlierBands:
    neighbours = np.append(maxs[band-2:band], maxs[band+1:band+3])
    newMaxValueBand = np.median(neighbours)
    bandList[band][bandList[band]>newMaxValueBand] = newMaxValueBand
    maxs[band] = newMaxValueBand

  if verbose:
    print('Output:')
    plt.figure(2)
    plt.plot(maxs)
    plt.show()

  return bandList

def stretch_hist(band, verbose = False):
  '''
  Removes the {percentage} least frequent pixel values from a band in order to decrease noise
  Admits a single band

  band = band.flatten()
  bs = math.ceil(band.max() - band.min())
  hist = plt.hist(band, bins=bs)[0]
  
  hist_s = np.array(sorted(hist))
  hist_sc = hist_s[hist_s>0]
  plt.figure()
  plt.hist(hist_sc)
  print(hist_sc)
  '''

  if isinstance(band, gdal.Dataset):
    band.ReadAsArray() # To numpy array

  band_no0 = band[band>0] # Mask will not be taken into account for the histogram stretching
  min_percent = 5   # Low percentile
  max_percent = 95  # High percentile

  # Find lower and upper percentile using percentile function, and "stretch" pixel range linearly between lower and upper percentiles
  lo, hi = np.percentile(band_no0, (min_percent, max_percent)) 

  # Apply linear "stretch" - lo goes to 0, and hi goes to 1
  res_img = (band.astype(float) - lo) / (hi-lo)

  #Multiply by 1, clamp range to [0, 1] and convert to float32
  res_img = np.maximum(np.minimum(res_img*1, 1), 0).astype(np.float32)

  if verbose:
    print('Input:')
    plt.figure(1)
    plt.imshow(band, cmap='gray', vmin=0, vmax=np.max(band))
    plt.show()
    print('Output (Normalized):')
    plt.figure(2)
    plt.imshow(res_img, cmap='gray', vmin=0, vmax=1)
    plt.show()

  return res_img


def normalize_bands(bandList):
  '''
  Fixes pixel values outliers in bands comparing to others
  Applies normalization between values [0-1]
  Admits both a list of bands and a single band
  '''
  if isinstance(bandList, list):
    bandList = band_max_values(bandList, 8)
    for i, band in enumerate(bandList):
      bandList[i] = stretch_hist(band)
    return bandList
  else:
    res_img = stretch_hist(bandList)
    return res_img

In [ ]:

# Landsat 7 from USGS cropped and gaps filled
dir_dst = os.getcwd() + '/drive/My Drive/TFG/Karnataka_20scenes/Landsat 7 Processed/LE071440522014062701T1-SC20201218115502/'
r = gdal.Open(dir_dst  + 'LE07_L1TP_144052_20140627_20161112_01_T1_sr_band3.tif', GA_Update).ReadAsArray()
g = gdal.Open(dir_dst  + 'LE07_L1TP_144052_20140627_20161112_01_T1_sr_band2.tif', GA_Update).ReadAsArray()
b = gdal.Open(dir_dst  + 'LE07_L1TP_144052_20140627_20161112_01_T1_sr_band1.tif', GA_Update).ReadAsArray()
plt.hist(r.flatten(), bins=200)
rgb = normalize_bands([r,g,b])
viz_raster(rgb[0], rgb[1], rgb[2])


# Hyperion from USGS
dir_dst = os.getcwd() + '/drive/My Drive/TFG/Karnataka_20scenes/EO-1 Hyperion/EO1H1440522014178110T2_1GST/EO1H1440522014178110T2/'
r = gdal.Open(dir_dst  + 'EO1H1440522014178110T2_B029_L1GST.TIF', GA_Update).ReadAsArray()
g = gdal.Open(dir_dst  + 'EO1H1440522014178110T2_B020_L1GST.TIF', GA_Update).ReadAsArray()
b = gdal.Open(dir_dst  + 'EO1H1440522014178110T2_B012_L1GST.TIF', GA_Update).ReadAsArray()
rgb = normalize_bands([r,g,b])
viz_raster(rgb[0], rgb[1], rgb[2])


In [ ]:
listdir_src = os.getcwd() + '/drive/My Drive/TFG/Ciudad Real/EO-1 ALI Processed Aligned/'
ALIdir_r = [os.path.join(listdir_src, f, f[:-5], f[:-4]+'B06_L1GST_modified3.tif') for f in sorted(os.listdir(listdir_src)[::-1])]
ALIdir_g = [os.path.join(listdir_src, f, f[:-5], f[:-4]+'B05_L1GST_modified3.tif') for f in sorted(os.listdir(listdir_src)[::-1])]
ALIdir_b = [os.path.join(listdir_src, f, f[:-5], f[:-4]+'B04_L1GST_modified3.tif') for f in sorted(os.listdir(listdir_src)[::-1])]
listdir_src = os.getcwd() + '/drive/My Drive/TFG/Ciudad Real/EO-1 Hyperion/'
HYPdir_r = [os.path.join(listdir_src, f, f[:-5], f[:-4]+'B215_L1GST.TIF') for f in sorted(os.listdir(listdir_src)[::-1])]
HYPdir_g = [os.path.join(listdir_src, f, f[:-5], f[:-4]+'B054_L1GST.TIF') for f in sorted(os.listdir(listdir_src)[::-1])]
HYPdir_b = [os.path.join(listdir_src, f, f[:-5], f[:-4]+'B018_L1GST.TIF') for f in sorted(os.listdir(listdir_src)[::-1])]

for i in range(len(ALIdir_r)):
  print('Scene',i, ALIdir_r[i])
  # ALI processed and coregistered from USGS Ciudad Real
  r = gdal.Open(ALIdir_r[i], GA_Update).ReadAsArray()
  g = gdal.Open(ALIdir_g[i], GA_Update).ReadAsArray()
  b = gdal.Open(ALIdir_b[i], GA_Update).ReadAsArray()
  rgb = normalize_bands([r,g,b])
  viz_raster(rgb[0], rgb[1], rgb[2])

  # Hyperion from USGS Ciudad Real
  r = gdal.Open(HYPdir_r[i], GA_Update).ReadAsArray()
  g = gdal.Open(HYPdir_g[i], GA_Update).ReadAsArray()
  b = gdal.Open(HYPdir_b[i], GA_Update).ReadAsArray()
  rgb = normalize_bands([r,g,b])
  viz_raster(rgb[0], rgb[1], rgb[2])


In [ ]:

# Landsat 7 from USGS
dir_dst = os.getcwd() + '/drive/My Drive/TFG/PaperReproduction/Landsat7India/LE071440522014062701T1-SC20201003173316/'
r = gdal.Open(dir_dst  + 'LE07_L1TP_144052_20140627_20161112_01_T1_sr_band3.tif', GA_Update).ReadAsArray()
g = gdal.Open(dir_dst  + 'LE07_L1TP_144052_20140627_20161112_01_T1_sr_band2.tif', GA_Update).ReadAsArray()
b = gdal.Open(dir_dst  + 'LE07_L1TP_144052_20140627_20161112_01_T1_sr_band1.tif', GA_Update).ReadAsArray()

plt.hist(r.flatten(), bins=200)
plt.show()
rgb = normalize_bands([r,g,b])
viz_raster(rgb[0], rgb[1], rgb[2])


#Landsat 7 from Earth Engine, not masked
dir_dst = os.getcwd() + '/drive/My Drive/TFG/Karnataka_20scenes/Landsat 7/Not masked/LE07_144052_20140627.tif'
im = gdal.Open(dir_dst, GA_Update)
r = im.GetRasterBand(3).ReadAsArray()
g = im.GetRasterBand(2).ReadAsArray()
b = im.GetRasterBand(1).ReadAsArray()

plt.hist(r.flatten(), bins=200)
plt.show()

rgb = normalize_bands([r,g,b])

plt.hist(rgb[0].flatten(), bins=200)
plt.show()
viz_raster(rgb[0], rgb[1], rgb[2])



#Landsat 7 from Earth Engine,  masked
dir_dst = os.getcwd() + '/drive/My Drive/TFG/Karnataka_20scenes/Landsat 7/Masked/LE07_144052_20140627.tif'
im = gdal.Open(dir_dst, GA_Update)
r = im.GetRasterBand(3).ReadAsArray()
g = im.GetRasterBand(2).ReadAsArray()
b = im.GetRasterBand(1).ReadAsArray()
plt.hist(r.flatten(), bins=200)
plt.show()
rgb = normalize_bands([r,g,b])
plt.hist(rgb[0].flatten(), bins=200)
plt.show()
viz_raster(rgb[0], rgb[1], rgb[2])

